# Obrada sekvenci pomocu RNN i CNN

## Rekurentni neuroni i slojevi

U svakom vremenskom koraku, rekurentni neuron prima ulazne podatke $x_{(t)}$ i vlastite izlaze iz prethodnog vremenskog koraka $y_{(t-1)}$

<figure style="display: inline-block">
  <img src="../img/2.6./recurrent_neuron.png" width="700" height="350">
  <figcaption style="text-align: center"></figcaption>
</figure>

Izlaz rekurentnog sloja jednog primera

$y_{(t)} = \phi(W_x^Tx_{(t)} + W_y^Ty_{(t-1)} + b)$

## Memorijske celije

Posto je izlaz rekurentnog neurona u vremenskom koraku t funkcija svih njegohiv ulaza iz prethodnih vremenskih koraka moze se reci da on ima nekakav oblik memorije. Deo neuronske mreze koji cuva odredjene podatke o stanju mreze tokom vremenskih koraka zove se memorijska celija. Jedan rekurentni neuron ili slojrekurentnih neurona je veoma jednostavan oblik celije koja je u stanju da nauci samo kratke obrasce.

## Ulazne i izlazne sekvence

Jedan RNN moze imati 4 konfiguracije ulaza i izlaza:
  1. <i>sekvenca u sekvencu</i> - mreza koja prima sekvencu ulaza i proizvodi sekvencu izlaza, korisna za serije podataka koje se menjaju tokom vremena
  2. <i>sekvenca u vektor</i> - mreza koja prima sekvencu ali zanemarimo sve njene izlaze sem poslednjeg
  3. <i>vektor u sekvencu</i> - mrezi se zadaje isti vektor ulaza ponovo i ponovo u svakom vremenskom koraku i prepusta se njoj da od njih generise sekvencu
  4. <i>sekvenca u vektor (enkoder), vektor u sekvencu (dekoder)</i> - ovo je mreza koja se sastoji od dva dela, prvi uzima sekvencu i pretvara je u neki oblik koji se daje dekoderu od koga on generise sekvencu 

## Obuka RNN

Trik kod obucavanja RNN je odmotavanje mreze kroz vremei da se zatim primeni standardna povratna propagacija.

<figure style="display: inline-block">
  <img src="../img/2.6./BPTT.png" width="600" height="350">
  <figcaption style="text-align: center"></figcaption>
</figure>

## Predvidjanje vremenskih serija

In [2]:
import tensorflow as tf
import keras
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [3]:
def generate_time_series(batch_size, n_steps):
  freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
  time = np.linspace(0, 1, n_steps)
  series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   wave 1
  series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
  series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + noise
  return series[..., np.newaxis].astype(np.float32)

In [4]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

### Osnovni merni pokazatelji

Najjednostavniji pristup je da se predvidi poslednja vrednost u svakoj seriji. To se zove naivno predvidjanje. U ovom slucaju postize vrednost srednje kvadratne greske 0.02.

In [6]:
y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.020211367

Drugi jednostavan pristup je koriscenje potpuno povezane mreze. Ovaj pristup postize vrednost od 0.004, znatno bolje.

In [7]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 1s 2ms/step - loss: 0.1001 - val_loss: 0.0545
Epoch 2/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0379 - val_loss: 0.0266
Epoch 3/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0202 - val_loss: 0.0157
Epoch 4/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0131 - val_loss: 0.0116
Epoch 5/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0103 - val_loss: 0.0098
Epoch 6/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0089 - val_loss: 0.0087
Epoch 7/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0080 - val_loss: 0.0079
Epoch 8/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0073 - val_loss: 0.0071
Epoch 9/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0066 - val_loss: 0.0066
Epoch 10/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0061 - val_loss: 0.0062

In [8]:
model.evaluate(X_valid, y_valid)

63/63 [==============================] - 0s 1ms/step - loss: 0.0042


0.004168086685240269

### Implementiranje jednostavne RNN

In [10]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(loss="mse", optimizer=optimizer)
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 2s 6ms/step - loss: 0.0967 - val_loss: 0.0489
Epoch 2/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0369 - val_loss: 0.0296
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0253 - val_loss: 0.0218
Epoch 4/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0198 - val_loss: 0.0177
Epoch 5/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0166 - val_loss: 0.0151
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0146 - val_loss: 0.0134
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0132 - val_loss: 0.0123
Epoch 8/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0124 - val_loss: 0.0116
Epoch 9/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0118 - val_loss: 0.0112
Epoch 10/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0116 - val_loss: 0.0110

In [11]:
model.evaluate(X_valid, y_valid)

63/63 [==============================] - 0s 2ms/step - loss: 0.0109


0.010881561785936356

### Duboke RNN

In [12]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 5s 16ms/step - loss: 0.0492 - val_loss: 0.0090
Epoch 2/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0070 - val_loss: 0.0065
Epoch 3/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0053 - val_loss: 0.0045
Epoch 4/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0045 - val_loss: 0.0040
Epoch 5/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0042 - val_loss: 0.0040
Epoch 6/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0038 - val_loss: 0.0036
Epoch 7/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0038 - val_loss: 0.0040
Epoch 8/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0037 - val_loss: 0.0033
Epoch 9/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0036 - val_loss: 0.0032
Epoch 10/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0035 - val_lo

In [13]:
model.evaluate(X_valid, y_valid)

63/63 [==============================] - 0s 5ms/step - loss: 0.0029


0.0029105644207447767